# Model-based aversive learning in humans is supported by preferential task state reactivation
Wise\*, Liu\*, Chowdhury, & Dolan (2021)

## MEG Preprocessing

#### _This is a template that will be parameterised and run via [Papermill](http://papermill.readthedocs.io/) for each subject_

This notebook performs preprocessing of localiser and task data. 

Preprocessing steps:

1. Identification and loading of raw data
2. Maxwell filtering
3. Filtering
4. ICA
5. Epoching
6. Downsampling


In [1]:
%pip install -r ../../requirements.txt

## Imports

In [15]:
from mne.io import read_raw_fif
import mne
import matplotlib.pyplot as plt
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
import numpy as np
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import os
import time
import datetime
import yaml
import papermill as pm
import pandas as pd
np.random.seed(100)
%matplotlib inline

## Parameters

In [22]:
# DEFAULT PARAMETERS - OVERRRIDEN BY PAPERMILL EXECUTION
data_dir = 'data/'  # Directory containing data
session_id = '001'  # ID of the scanning session
n_runs = 12  # Number of runs
eye_tracking = True  # If True, eye-tracking measures will be used for exclusion of blink-related ICA components
maxwell = True # If true, use maxwell filtering to clean data
filter_low = 0.5 # Band pass lower freq
filter_high = 0.5 # Band pass upper freq
n_stim = 14  # Number of stimuli
cores = 1  # Number of cores to use for parallel processing
blink_components = None  # ICA components to remove
downsample = True
os.environ['OMP_NUM_THREADS'] = str(cores)

In [23]:
if filter_low == "None":
    filter_low = None
if filter_high == "None":
    filter_high = None

In [24]:
output_dir = 'sub-{0}'.format(session_id)  # Where the output data should go

## Get data

Data is stored in [BIDS format](https://www.nature.com/articles/sdata2018110) - when I wrote this MNE didn't directly read from BIDS however.

In [25]:
# Set the data directory for this subject
data_dir = os.path.join(data_dir, 'sub-{0}'.format(session_id), 'ses-01', 'meg')

# Find all files in the directory and make sure they're in the right order (i.e. ascending)
data = os.listdir(data_dir)
data = sorted([i for i in data if '.ds' in i and str(session_id) in i and not 'opt' in i])

# Check we have the right number of runs
assert len(data) == n_runs, "Wrong number of data files, found {0}".format(len(data))

# See what has been found
print(data)

# Get all the data and read it in
raws = []
run_idx = range(0, n_runs)

# Read in each data set
for i in run_idx:
    start_time = time.time()
    raws.append(read_raw_fif(os.path.join(data_dir, data[i]), preload=True))
    time_taken = time.time() - start_time
    print("Time taken = {0}".format(str(datetime.timedelta(seconds=time_taken))))

# Concatenate the runs
raw = mne.concatenate_raws(raws)

# Get events
print("FINDING EVENTS")
events = mne.find_events(raw, stim_channel='UPPT001', shortest_event=1)
events[:, 0] += int((1 / 60 * 2) * raw.info['sfreq'])  # Adjust event times to compensate for projector lag (2 frames)

del raws  # delete the list of raw data to conserve memory

# We recorded at 1200hz, but this makes everything take FOREVER so we downsample to 600hz
raw, events = raw.copy().resample(600, npad='auto', events=events)

# Label eye-tracking channels as EOG
raw.set_channel_types({'UADC001-2910': 'eog', 'UADC002-2901': 'eog', 'UADC003-2901': 'eog'})

AssertionError: Wrong number of data files, found 0

## Preprocessing

### Apply Maxwell filter
This is partly because some MNE functions don't seem to work properly if CTF compensation is turned on, however Maxwell filtering also appears to do a better job than CTF compensation at removing noise from movement etc (https://martinos.org/mne/stable/auto_tutorials/plot_brainstorm_phantom_ctf.html)

In [ ]:
if maxwell:
    raw.apply_gradient_compensation(0)  # Remove CTF compensation
    mf_kwargs = dict(origin=(0., 0., 0.), st_duration=10.)
    raw = mne.preprocessing.maxwell_filter(raw, **mf_kwargs)

### Plot the raw data

In [ ]:
raw.plot()

### Filter

Highpass filter above 0.5hz, using windowed FIR filter with MNE default settings.

In [ ]:
print("FILTERING")
raw.filter(filter_low, filter_high, method='fir', fir_design='firwin')

## ICA

ICA is performed on the raw data and is set to find the number of components that explains 95% of the variance. 

We don't do much in terms of selecting noise-related components here - the data is generally pretty clean and doesn't seem to benefit much from extra denoising, so we simply automatically detect blink-related components based on eye-tracking channels.

For some subjects eye tracking was poor due to equipment problems. If we're not able to identify a blink-related component automatically that may be due to an absent eye tracking channel, or just due to noisy eye tracking data - in this case the notebook uses provided components (these were identified manually in a previous run).

In [ ]:
# Run ICA
picks_meg = mne.pick_types(raw.info, meg=True, ref_meg=False)
reject = dict(mag=5e-12, grad=4000e-13)
ica = ICA(n_components=0.95, method='fastica',
          random_state=100, max_iter=100).fit(raw, decim=50, picks=picks_meg, reject=reject)

# Plot components
ica.plot_components()

# Save decomposition
if not os.path.exists(os.path.join(output_dir, 'ICA')):
    os.makedirs(os.path.join(output_dir, 'ICA'))
ica.save(os.path.join(output_dir, 'ICA', 'sub-{0}_ses-01_task-AversiveLearningReplay_proc-ICA.fif.gz').format(session_id))

# Find blink-related components
if blink_components is None or blink_components == 'None':
    blink_components, scores = ica.find_bads_eog(raw, threshold=1.5)
    ica.plot_scores(scores, exclude=blink_components, labels='blink')
    show_picks = np.abs(scores).argsort()[::-1][:5]
    ica.plot_components(blink_components, colorbar=True)
    
# Find ECG componenhts
ecg_epochs = create_ecg_epochs(raw, tmin=-.5, tmax=.5, picks=picks_meg)

ecg_components, scores = ica.find_bads_ecg(ecg_epochs, method='ctps')
ica.plot_scores(scores, exclude=ecg_components, labels='ecg')
ecg_components = ecg_components[:3]

print("APPLYING ICA")

# Only select a maximum of 2 components
blink_components = blink_components[:2]

ica.exclude = blink_components + ecg_components
ica.apply(raw)

### Create epochs

Here we split the continuous data into epochs. Stimulus triggers are from 2 to the number of stimuli * 2 with a step of 2 (this is because sending odd numbers also triggers shocks in the actual task). Code 99 is used for null trials (only used in the localiser). We don't reject any trials because we want to decide on rejections later.

We're only selecting the planning and rest periods from the task here as these are the periods we'll be looking at in later analyses.

In [ ]:
## CATCH DUPLICATE EVENTS DUE TO PAUSES

duplicate_idx = np.where(np.diff(events[:, 2]) == 0)[0]

if np.any(events[duplicate_idx] == 60):
    delete_idx = duplicate_idx[events[duplicate_idx, 2] == 60][0]
    events = np.delete(events, delete_idx, axis=0)

Behavioural data was mislabelled for one subject, this means the stimuli used for localiser and task aren't the same so we need to fix this!

In [ ]:
if session_id == '001':
    import re

    beh_data_dir = os.path.join(data_dir, 'sub-{0}'.format(session_id), 'ses-01', 'beh')

    # OLD - NEED TO MOVE BEHAVIOURAL FILES
    localiser_stimuli_file = os.path.join('localiser/Data', [i for i in os.listdir('localiser/Data') if session_id in i and 'stimuli' in i][0])
    task_stimuli_file = os.path.join('task/Data/behavioural/', [i for i in os.listdir('task/Data/behavioural/') if '001' in i and 'stimuli' in i][0])

    # NEW 
    localiser_stimuli_file = os.path.join(beh_data_dir, 'sub-{0}_ses-01_task-AversiveLearningReplay_localiser-stim.csv'.format(session_id))
    task_stimuli_file = os.path.join(beh_data_dir, 'sub-{0}_ses-01_task-AversiveLearningReplay_task-stim.csv'.format(session_id))
    
    def get_stimuli(log_file):
        with open(log_file, 'r') as f:
            stimuli = f.read().split(',')
        stimuli = [re.search('[0-9]{2}', i).group() for i in stimuli]
        return stimuli

    def match_stimuli(localiser, task):
        localiser_stimuli = get_stimuli(localiser)
        task_stimuli = get_stimuli(task)
        new_idx = [localiser_stimuli.index(i) for i in task_stimuli]
        return new_idx

    # This returns the correct indices for the task stimuli
    correct_idx = match_stimuli(task_stimuli_file, localiser_stimuli_file)
    
    localiser_events = np.arange(2, n_stim * 2 + 2, 2)
    
    original_events = events.copy()
    
    for n, i in enumerate(localiser_events):
        events[original_events[:, 2] == i, 2] = localiser_events[correct_idx][n]
    

Now we get the epochs

In [ ]:
print("EPOCHING")

# Get event names
# Event numbers for the localiser are even numbers from 2 up to 2 * the number of stimuli
localiser_event_names = dict([('stimulus_{0}'.format(i), i) for i in list(range(2, n_stim * 2 + 1, 2))])

# We get task event numbers from the task config file
with open('settings/replay_task_settings.yaml', 'rb') as f:
    task_config = yaml.load(f)
task_event_names = task_config['triggers']
    
# Occasionally planning triggers get coded as 62 instead of 60, so change any 62s to 60s. This might be just in cases with the incorrect trigger timing (M200-203)
events[:, 2][events[:, 2] == 62] = 60
events[:, 2][events[:, 2] == 98] = 99 # 98 for null events for one subject these got recorded as 98 rather than 99 for no apparent reason

# Create the epoch objects
localiser_epochs = mne.Epochs(raw, events[np.isin(events[:, 2], list(localiser_event_names.values()))], tmin=-0.5, tmax=0.8, preload=True, event_id=localiser_event_names,
                              reject=None)
planning_epochs = mne.Epochs(raw, events[np.isin(events[:, 2], list(task_event_names.values()))], 
                         tmin=0, tmax=np.max([task_config['MEG_durations']['start_duration'], task_config['MEG_durations']['rest_duration']]), 
                         preload=True, event_id={'planning': task_event_names['planning']}, reject=None)

# Final state shown for 1.2 seconds, outcome for 2.8 seconds
outcome_epochs = mne.Epochs(raw, events[np.isin(events[:, 2], list(task_event_names.values()))], 
                         tmin=-1.2, tmax=task_config['MEG_durations']['move_durations'][-1] - task_config['MEG_durations']['shock_symbol_delay'], 
                         preload=True, event_id={k: task_event_names[k] for k in ('shock_outcome', 'no_shock_outcome')}, reject=None)


In [ ]:
print("LOCALISER EVENTS")

for k, v in localiser_event_names.items():
    print("Number of {0} events = {1}".format(k, np.sum(events[:, 2] == v)))
    
print("TASK EVENTS")

for k, v in task_event_names.items():
    print("Number of {0} events = {1}".format(k, np.sum(events[:, 2] == v)))

### Plot the events

In [ ]:
data = []

for n, i in enumerate(np.unique(events[:, 2])):
    data.append(go.Scatter(x=events[events[:, 2] == i][:, 0], y=[n] * len(events[events[:, 2] == i][:, 0]), mode = 'markers'))
    

layout = dict(title='Events', showlegend=False,
              xaxis=dict(title='Samples'), yaxis=dict(title='Event ID', tickvals=np.arange(len(np.unique(events[:, 2]))), ticktext=[str(i) for i in np.unique(events[:, 2])]), 
              width=1500, height=600)

fig = dict(data=data, layout=layout)
iplot(fig, filename='events')


### Check the events look right

First, we should have at least 700 localiser events 

In [ ]:
assert len(localiser_epochs) > 700, 'Unexpected number of localiser trials, found {0}, expected 890'.format(len(localiser_epochs))

### Downsample

Resampling to 100hz.

In [ ]:
del raw  # Delete the raw data variable to save memory

if downsample:
    print("DOWNSAMPLING")
    print('Original sampling rate: {0} Hz'.format(localiser_epochs.info['sfreq']))
    localiser_epochs = localiser_epochs.copy().resample(100, npad='auto')
    planning_epochs = planning_epochs.copy().resample(100, npad='auto')
    outcome_epochs = outcome_epochs.copy().resample(100, npad='auto')
    print('Downsampled sampling rate: {0} Hz'.format(localiser_epochs.info['sfreq']))

### Save the epoched data

In [ ]:
localiser_epochs.save(os.path.join(output_dir, 'localiser', 'sub-{0}_ses-01_task-AversiveLearningReplay_run-localiser_proc_ICA-epo.fif.gz').format(session_id))
planning_epochs.save(os.path.join(output_dir, 'task', 'sub-{0}_ses-01_task-AversiveLearningReplay_run-planning_proc_ICA-epo.fif.gz').format(session_id))
outcome_epochs.save(os.path.join(output_dir, 'task', 'sub-{0}_ses-01_task-AversiveLearningReplay_run-outcome_proc_ICA-epo.fif.gz').format(session_id))